## How to query a Nexus SPARQL View ?

This tutorial demonstrates how to:
* run a query against an existing Nexus Sparql View
* convert the sparql result to a pandas data frame


In [ ]:
#Let install sparqlwrapper which a python wrapper around sparql client
!pip install git+https://github.com/RDFLib/sparqlwrapper

In [ ]:
import pandas as pd
from itertools import repeat
import concurrent.futures
import json
import csv
import requests
import time
from urllib.parse import urlencode, quote_plus
from collections import OrderedDict
import pandas as pd
from io import StringIO
from functools import reduce
from SPARQLWrapper import SPARQLWrapper, XML, N3, TURTLE, JSON, POST, POSTDIRECTLY, CSV

In [ ]:
#Configure a sparql client for a Nexus Sparql View

token ="eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJONS1CU0ZxZG5NS3Y4SWtKUkg1R3E0LVA2c1RWQUxwU0EydGNQeEpWM1NBIn0.eyJqdGkiOiI3ZjhlZTJiMC1mOWQ5LTQ4ZmEtOWI4MC04ZDQ3YTQ1MzMzMzIiLCJleHAiOjE1NDg3NTM2NzYsIm5iZiI6MCwiaWF0IjoxNTQ4MTQ4ODc2LCJpc3MiOiJodHRwczovL2JicHRlYW0uZXBmbC5jaC9hdXRoL3JlYWxtcy9CQlAiLCJhdWQiOiJiYnAtbmV4dXMtcHJvZHVjdGlvbiIsInN1YiI6ImY6OWQ0NmRkZDYtMTM0ZS00NGQ2LWFhNzQtYmRmMDBmNDhkZmNlOnN5IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYmJwLW5leHVzLXByb2R1Y3Rpb24iLCJhdXRoX3RpbWUiOjE1NDgxNDg4NzYsInNlc3Npb25fc3RhdGUiOiJmYjgxNmI3Mi1iZTVkLTQ4MzEtYmI3My03YWY1NjNjOWNkMmMiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbIi8qIl0sInJlc291cmNlX2FjY2VzcyI6e30sInNjb3BlIjoib3BlbmlkIG5leHVzIiwibmFtZSI6Ik1vaGFtZXRoIEZyYW7Dp29pcyBTeSIsInByZWZlcnJlZF91c2VybmFtZSI6InN5IiwiZ2l2ZW5fbmFtZSI6Ik1vaGFtZXRoIEZyYW7Dp29pcyIsImZhbWlseV9uYW1lIjoiU3kiLCJlbWFpbCI6Im1vaGFtZXRoLnN5QGVwZmwuY2gifQ.lquFwadTt3aK0U92zkH4So2vu5Ip36g2jnirX2NWYuTDbV_iCQ9RkCWIvLwUzGM9C0MXaSIWhcD0jRZFyIdyFFtrEN3nKE-lXGObt1aFZSnoikF8xOuo9WqcPYfycjePeD3tOeRmSGBIGGbzxceZGQ5VFLya3Vpd2olI6GS-XmNCPYK0IzjekqiR7m4F4RxcoZkh1_Fh8qamuftV-G2H-l9T6aGcss_sIeH9pu1dd4bkJoq9xveVP3c69aWlaoGSHX_YU7DuzmTnx8u1iJ8hjztg1rD858yQBy4qjJO743awZfRKE0WJL8o-pIzGnzM2_5BvMJwYZ14Is-dnYcqb-Q"
ns = "https://bbp.epfl.ch/nexus/v1/"
endpoint = ns+"views/testamld2019/movielens2/graph/sparql/"
headers = {}
headers["Authorization"] ="Bearer {}".format(token)
headers["Content-Type"] ="application/sparql-query"

sparql_client = SPARQLWrapper(endpoint)
sparql_client.addCustomHttpHeader("Content-Type", "application/sparql-query")
sparql_client.addCustomHttpHeader("Authorization","Bearer {}".format(token))
sparql_client.setMethod(POST)
sparql_client.setReturnFormat(JSON)
sparql_client.setRequestMethod(POSTDIRECTLY)

In [ ]:
# Convert SPARQL results into a Pandas data frame
def sparql2dataframe(json_sparql_results):
    cols = json_sparql_results['head']['vars']
    out = []
    for row in json_sparql_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

#Use a client and send a query
def query(query, sparql_client):
    sparql_client.setQuery(query)
    result_object = sparql_client.query()
    return result_object._convertJSON()
   
    

In [ ]:
# Configure and run query

querystring = """
PREFIX vocab: <$NEXUS-URL/vocabs/tutorialnexus/$PROJECTLABEL/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>

Select ?userId ?movieId  ?rating
 WHERE  {
    ?ratingNode a vocab:Rating.
    ?ratingNode vocab:movieId ?movieId.
    ?ratingNode vocab:rating ?rating.
    ?ratingNode vocab:userId ?userId.

}
ORDER BY ASC(?userId)
LIMIT 250"""



results = query(querystring,sparql_client)

# Convert to a dataframe

df = sparql2dataframe(results)
df.head()





